# Notebook para explorar datos y definir acciones a ejecutar en transform.py

In [15]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Cargar dfs
df_bibliotecas_path = '../data/bibliotecas/2022-octubre/bibliotecas-07-10-2022.csv'
df_cines_path = '../data/cines/2022-octubre/cines-07-10-2022.csv'
df_museos_path = '../data/museos/2022-octubre/museos-07-10-2022.csv'

bibliotecas_df = pd.read_csv(df_bibliotecas_path, encoding='utf8')
cines_df = pd.read_csv(df_cines_path, encoding='utf8')
museos_df = pd.read_csv(df_museos_path, encoding='utf8')

# Ver tamaños y columnas de cada uno
dfs = [bibliotecas_df, cines_df, museos_df]
for df in dfs:
    display(df.columns)
    display(df.shape)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observacion', 'Categoría',
       'Subcategoria', 'Provincia', 'Departamento', 'Localidad', 'Nombre',
       'Domicilio', 'Piso', 'CP', 'Cod_tel', 'Teléfono', 'Mail', 'Web',
       'Información adicional', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'Tipo_gestion', 'año_inicio', 'Año_actualizacion'],
      dtype='object')

(2017, 25)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'Categoría', 'Provincia', 'Departamento', 'Localidad', 'Nombre',
       'Dirección', 'Piso', 'CP', 'cod_area', 'Teléfono', 'Mail', 'Web',
       'Información adicional', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'tipo_gestion', 'Pantallas', 'Butacas', 'espacio_INCAA',
       'año_actualizacion'],
      dtype='object')

(329, 26)

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'categoria', 'subcategoria', 'provincia', 'localidad', 'nombre',
       'direccion', 'piso', 'CP', 'cod_area', 'telefono', 'Mail', 'Web',
       'Latitud', 'Longitud', 'TipoLatitudLongitud', 'Info_adicional',
       'fuente', 'jurisdiccion', 'año_inauguracion', 'actualizacion'],
      dtype='object')

(1182, 24)

## A) Transformaciones para master dataset
### 1° Unificar nombres de columnas comunes requeridas entre los 3 dfs

In [16]:
# Columnas para master_df: cod_localidad, id_provincia, id_departamento, categoría, 
# provincia, localidad, nombre, domicilio, código postal, número de teléfono, mail, web

# Cargar nombres de columnas de interés de cada df en mismo orden
columns = ['cod_localidad', 'id_provincia', 'id_departamento', 'categoria', 
                'provincia', 'localidad','nombre','domicilio', 'codigo_postal', 'Cod_tel',
                'Teléfono', 'mail', 'web' ]
bibliotecas_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Categoría', 
                        'Provincia', 'Localidad','Nombre','Domicilio', 'CP', 'Cod_tel', 
                        'Teléfono', 'Mail', 'Web']
cines_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Categoría', 'Provincia', 
                    'Localidad','Nombre','Dirección', 'CP', 'cod_area', 'Teléfono', 'Mail', 
                    'Web']
museos_columns = ['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'categoria', 'provincia', 
                    'localidad','nombre','direccion', 'CP', 'cod_area', 'telefono', 
                    'Mail', 'Web' ]

# Renombrar columnas en cada df para que todos sean iguales y se pueda concatenar
bibliotecas_df.rename(columns=dict(zip(bibliotecas_columns, columns)), inplace=True)
cines_df.rename(columns=dict(zip(cines_columns, columns)), inplace=True)
museos_df.rename(columns=dict(zip(museos_columns, columns)), inplace=True)

# Concatenar dfs en master y ver resultados
master_df = pd.concat([bibliotecas_df[columns], cines_df[columns], museos_df[columns]])
display(master_df.columns, master_df.shape)
master_df.sample(5)

Index(['cod_localidad', 'id_provincia', 'id_departamento', 'categoria',
       'provincia', 'localidad', 'nombre', 'domicilio', 'codigo_postal',
       'Cod_tel', 'Teléfono', 'mail', 'web'],
      dtype='object')

(3528, 13)

,cod_localidad,id_provincia,id_departamento,categoria,provincia,localidad,nombre,domicilio,codigo_postal,Cod_tel,Teléfono,mail,web
621,6098010,6,6098,Bibliotecas Populares,Buenos Aires,Villa B. Constructor,Biblioteca Popular Belisario Roldán,15 4211,1923.0,221.0,4611872.0,s/d,NaN
1467,6805010,6,6805,Bibliotecas Populares,Buenos Aires,El Talar de Pacheco,Biblioteca Popular Lopez Camelo,Paul Groussac 3302,1618.0,3327.0,444949.0,cultura@cotelnet.com.ar,NaN
1825,82014140,82,82014,Bibliotecas Populares,Santa Fe,Villada,Asociación Civil Biblioteca Popular Mariano Moreno de Villada,San Miguel 376,2173.0,341.0,4257166.0,bibpopmarianomoreno@yahoo.com.ar,NaN
108,6672010,6,6672,Espacios de Exhibición Patrimonial,Buenos Aires,Rauch,Museo Municipal De Artes Plásticas Faustino Jorge Bonadeo,Avda. San Martín 155,B7203BPB,2297.0,443 249,NaN,NaN
1102,6441030,6,6441,Bibliotecas Populares,Buenos Aires,La Plata,Biblioteca Pública Central General José de San Martín,Calle 47 510,1900.0,221.0,4254954.0,bibliotecacentraldelaprovincia@yahoo.com.ar,NaN


### 2° Unificar cod_tel y telefono en una única columna

In [17]:
display(master_df['Cod_tel'].dtype, master_df['Teléfono'].dtype)
master_df[['Cod_tel', 'Teléfono']].sample(9)

dtype('float64')

dtype('O')

,Cod_tel,Teléfono
637,NaN,NaN
1027,2652.0,459826.0
82,2323.0,420 755
1544,3876.0,422739.0
539,3442.0,426 540
809,358.0,4882335.0
1953,351.0,6006861.0
23,11.0,46641840.0
753,261.0,941 050


In [18]:
# Cod_tel es de tipo float por lo que los nros tienen agregado .0 al final, y hay valores nulos
# Teléfono es tipo str, tiene valores con espacios en blanco, pero también tiene algunos con .0 al final y valores nulos
# La nueva columna será str para preservar nulos, y se creará uniendo los dos campos (siempre y cuando haya valor en Teléfono)
# separados por un espacio (para seguir formato presentado), previa eliminación de los .0

def make_phone_number(row):

    if not pd.isnull(row['Teléfono']):
        # convierto a str
        tel = str(row['Teléfono']).strip()
        # elimino .0 final si tiene
        if tel.find('.0') != -1:
            tel = tel[:tel.find('.0')]
        # chequeo código para ver si debo concatenar o va solo nro de telefono sin cod area
        if not pd.isnull(row['Cod_tel']):
            cod = str(row['Cod_tel']).strip()
            if cod.find('.0') != -1:
                cod = cod[:cod.find('.0')]
            row['telefono'] = cod + ' ' + tel
        else:
            row['telefono'] = tel
    else:
    # si no tengo número de teléfono no me interesa si hay código de área 
        row['telefono'] = np.nan
    return row    

master_df = master_df.apply(make_phone_number, axis='columns')

master_df[['Cod_tel', 'Teléfono', 'telefono']].sample(12)
master_df.drop(['Cod_tel', 'Teléfono'], inplace=True, axis='columns')
columns.remove('Cod_tel')
columns.remove('Teléfono')

### Explorar datos por columna

In [19]:
# Explorar datos, chequeando nulos, valores únicos y distribución de valores
total_rows = master_df.shape[0]
for column in columns:
    display(f'---------------------------- Columna {column} - {master_df[column].dtype} ----------------------------')
    no_nulos = master_df[column].count()
    unique_values_count = len(master_df[column].unique())
    display(f'Total: {total_rows}  |  Nulos: {total_rows - no_nulos}  |  No nulos: {no_nulos} | Únicos: {unique_values_count}')
    if unique_values_count > 30:
        display(f'Top Distribución de valores:')
        display(master_df[column].value_counts().iloc[:15])
    else:
        display(f'Valores únicos:')
        display(master_df[column].unique())
        display(f'Distribución de valores:')
        display(master_df[column].value_counts())


'---------------------------- Columna cod_localidad - int64 ----------------------------'

'Total: 3528  |  Nulos: 0  |  No nulos: 3528 | Únicos: 1197'

'Top Distribución de valores:'

2000010     210
14014010     92
82063170     72
6441030      63
82084270     61
66028050     50
22140060     46
58035070     37
86049110     31
50007010     31
70028010     30
90084010     27
6056010      24
6427010      22
38021060     21
Name: cod_localidad, dtype: int64

'---------------------------- Columna id_provincia - int64 ----------------------------'

...

s/d                                         561
rmazzutti@hoyts.com.ar                       11
iventura@cinemark.com.ar                      9
info@atlascines.com.ar                        5
fharari@hotmail.com.ar                        4
diversidad.cultural@santafeciudad.gov.ar      4
admin@lastipasboulevard.com.ar                3
nuevocineteatrorex@gmail.com                  3
cluduena@grupodinosaurio.com                  3
rabdenur@gmail.com                            3
ericontadoralinares@gmail.com                 3
naiarg@naiarg.com.ar                          3
gabriel.guralnik@gmail.com                    2
york.cultura@vicentelopez.gov.ar              2
museoesperanza@esperanza.gov.ar               2
Name: mail, dtype: int64

'---------------------------- Columna web - object ----------------------------'

'Total: 3528  |  Nulos: 3124  |  No nulos: 404 | Únicos: 308'

'Top Distribución de valores:'

http://www.cinemacenter.com.ar/            17
s/d                                        16
https://www.cinemarkhoyts.com.ar/          11
https://www.cinemarkhoyts.com.ar           10
https://www.villagecines.com/               8
cineatlasweb.com.ar                         7
https://www.multiplex.com.ar/               6
https://www.todoshowcase.com/               6
http://www.cinesunstar.com/                 6
www.ara.mil.ar                              4
www.enteculturaltucuman.gov.ar              4
http://cinema3dlarioja.com/                 3
http://www.cinemalaplata.com                3
www.culturarionegro.gov.ar                  3
https://www.cinesdinomall.com.ar/Inicio     2
Name: web, dtype: int64

#### Resultados de de la exploración
- IdProvincia tiene 24 valores únicos pero Provincia tiene 27 valores únicos. Hay 3 provincias que tienen dos nombres cada una y se deben unificar: 
1. Neuquén y Neuquén\xa0, 
2. Tierra del Fuego y Tierra del Fuego, Antártida e Islas del Atlántico Sur
3. Santa Fe y Santa Fé
- Domicilio: tiene 18 nulos + 30 'sin dirección', convertir a nulos
- Web tiene 3124 nulos y 16 valores 's/d', convertir a nulos
- Mail tiene 918 nulos y 561 's/d', convertir a nulos
- CP es string pero tiene valores con .0 al final, eliminar .0

### 3° Unificar nombres de Provincia

In [20]:
master_df[['id_provincia', 'provincia', 'categoria']].groupby(['id_provincia', 'provincia']).count().sort_values(by='id_provincia')

,,categoria
id_provincia,provincia,
2,Ciudad Autónoma de Buenos Aires,210
6,Buenos Aires,801
10,Catamarca,74
14,Córdoba,346
18,Corrientes,80
22,Chaco,107
26,Chubut,102
30,Entre Ríos,128
34,Formosa,34


In [21]:
def unify_province_names(row):
    if row['provincia'] == 'Neuquén\xa0':
        row['provincia'] = 'Neuquén'
    elif row['provincia'] == 'Santa Fe':
        row['provincia'] = 'Santa Fé'
    elif row['provincia'] == 'Tierra del Fuego':
        row['provincia'] = 'Tierra del Fuego, Antártida e Islas del Atlántico Sur'
    return row
master_df = master_df.apply(unify_province_names, axis='columns')

master_df[['id_provincia', 'provincia', 'categoria']].groupby(['id_provincia', 'provincia']).count().sort_values(by='id_provincia')

,,categoria
id_provincia,provincia,
2,Ciudad Autónoma de Buenos Aires,210
6,Buenos Aires,801
10,Catamarca,74
14,Córdoba,346
18,Corrientes,80
22,Chaco,107
26,Chubut,102
30,Entre Ríos,128
34,Formosa,34


- Se detecta una row que tiene Salta con codigo 58, que corresponde a Neuquén. Cambiar a 66
### 4° Cambiar cód de Salta en row erronea

In [22]:
# Chequear si lo que está incorrecto es el codigo o el nombre de la pcia.
master_df.loc[(master_df['provincia'] == 'Salta') & (master_df['id_provincia'] == 58)]

,cod_localidad,id_provincia,id_departamento,categoria,provincia,localidad,nombre,domicilio,codigo_postal,mail,web,telefono
860,58035070,58,58035,Espacios de Exhibición Patrimonial,Salta,Salta,Museo Catedralicio,Av. Belgrano 589,NaN,NaN,NaN,NaN


In [23]:
master_df.loc[(master_df['provincia'] == 'Salta') & (master_df['id_provincia'] == 58), 'id_provincia'] = 66

#chequear con group by
master_df[['id_provincia', 'provincia', 'categoria']].groupby(['id_provincia', 'provincia']).count().sort_values(by='id_provincia')

,,categoria
id_provincia,provincia,
2,Ciudad Autónoma de Buenos Aires,210
6,Buenos Aires,801
10,Catamarca,74
14,Córdoba,346
18,Corrientes,80
22,Chaco,107
26,Chubut,102
30,Entre Ríos,128
34,Formosa,34


- Antes de unificar nulos, chequear que no haya 0s o espacios en blanco en ninguna columna

In [24]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

columns.append('telefono')

for column in columns:
    if is_string_dtype(master_df[column]):
        # Para str chequeo espacio en blanco y vacio
        empty_count = master_df.loc[master_df[column] == '', column].count()
        whitespace_count = master_df.loc[master_df[column] == ' ', column].count()        
        display(f'{column}: empty {empty_count}, whitespace {whitespace_count}')
    elif is_numeric_dtype(master_df[column]):
        # Para numeric chequeo 0
        zero_count = master_df.loc[master_df[column] == 0, column].count()
        display(f'{column}: zero {zero_count}')

'cod_localidad: zero 0'

'id_provincia: zero 0'

'id_departamento: zero 0'

'categoria: empty 0, whitespace 0'

'provincia: empty 0, whitespace 0'

...

'domicilio: empty 0, whitespace 0'

'codigo_postal: empty 0, whitespace 0'

'mail: empty 0, whitespace 0'

'web: empty 0, whitespace 0'

'telefono: empty 0, whitespace 0'

### 5° Unificar nulos

In [25]:
#Domicilio: tiene 18 nulos + 30 'Sin dirección', convertir a nulos
#Web tiene 3124 nulos y 16 valores 's/d', convertir a nulos
#Mail tiene 918 nulos y 561 's/d', convertir a nulos

def convert_text_nulls_to_nan(row):
    nulls = ['sin dirección', 's/d']
    if str(row['domicilio']).strip().lower() in nulls:
        row['domicilio'] = np.nan
    if str(row['web']).strip().lower() in nulls:
        row['web'] = np.nan
    if str(row['mail']).strip().lower() in nulls:
        row['mail'] = np.nan
    return row

master_df = master_df.apply(convert_text_nulls_to_nan, axis='columns')
# Check
display(master_df['domicilio'].value_counts())
display(master_df['mail'].value_counts())
display(master_df['web'].value_counts())



Sarmiento               10
San Martín               9
25 de mayo               5
9 de julio               5
Sarmiento 450            4
                        ..
Junín 2238               1
Chubut y O.del Sur       1
Ruta 36 Km 3             1
Av. José Roberti 363     1
Ernesto Padilla S/N      1
Name: domicilio, Length: 3367, dtype: int64

rmazzutti@hoyts.com.ar                      11
iventura@cinemark.com.ar                     9
info@atlascines.com.ar                       5
diversidad.cultural@santafeciudad.gov.ar     4
fharari@hotmail.com.ar                       4
                                            ..
dirrecioncultura@52.coopenet.com.ar          1
biblio2945amenabar@hotmail.com               1
bibliotecahoracioquiroga@live.com            1
bibliotecalugones@bombal.net.ar              1
museoejercitotucuman@hotmail.com             1
Name: mail, Length: 1978, dtype: int64

http://www.cinemacenter.com.ar/                                                       17
https://www.cinemarkhoyts.com.ar/                                                     11
https://www.cinemarkhoyts.com.ar                                                      10
https://www.villagecines.com/                                                          8
cineatlasweb.com.ar                                                                    7
                                                                                      ..
https://twitter.com/cineteatroalfa                                                     1
https://www.facebook.com/pg/CineAHZ/about/?ref=page_internal                           1
https://www.facebook.com/pg/Cines-Avenida-112830095434025/about/?ref=page_internal     1
https://www.facebook.com/cine.italia/                                                  1
www.casaindependencia.com.ar                                                           1
Name: web, Length: 30

### 6° Quitar .0 de CP

In [26]:
display(master_df['codigo_postal'].dtype)
master_df['codigo_postal'].sample(6)

dtype('O')

863     A4427AQP
753     M5533XAZ
679       5139.0
997         2132
1315      8353.0
1144    V9420LCK
Name: codigo_postal, dtype: object

In [27]:
master_df['codigo_postal'] = master_df['codigo_postal'].map(lambda x: str(x)[:-2] if str(x).find('.0') != -1 else x)
master_df['codigo_postal'].sample(6)

109         3572
128         8103
1535         NaN
1018    S2300LND
1142        2535
782         4220
Name: codigo_postal, dtype: object

## B) Transformaciones para resumen dataset

### 1° Cantidad de registros totales por categoria

In [59]:
# Cantidad de registros totales por categoría
df_totales_categoria = master_df[['id_provincia', 'categoria']].groupby('categoria').count()
df_resumen = pd.DataFrame(
    {
        'etiqueta': ['categoria']*3,
        'valor': df_totales_categoria.index,
        'total': df_totales_categoria['id_provincia']
    }
)

df_resumen.reset_index(inplace=True)
df_resumen.drop(['categoria'], inplace=True, axis='columns')
df_resumen

,etiqueta,valor,total
0,categoria,Bibliotecas Populares,2017
1,categoria,Espacios de Exhibición Patrimonial,1182
2,categoria,Salas de cine,329


### 2° Cantidad de registros totales por fuente

In [60]:
# Cantidad de registros totales por fuente

display(f'Bibliotecas: {bibliotecas_df.shape[0]}')
display(f'Cines: {cines_df.shape[0]}')
display(f'Museos: {museos_df.shape[0]}')
df_resumen2 = pd.DataFrame(
    {
        'etiqueta': ['fuente']*3,
        'valor': ['bibliotecas', 'cines', 'museos'],
        'total': [bibliotecas_df.shape[0], cines_df.shape[0], museos_df.shape[0]]
    } 
)
df_resumen2

'Bibliotecas: 2017'

'Cines: 329'

'Museos: 1182'

,etiqueta,valor,total
0,fuente,bibliotecas,2017
1,fuente,cines,329
2,fuente,museos,1182


### 3° Cantidad de registros por provincia y categoria

In [56]:
# Cantidad de registros por provincia y categoría
df_totales_prov_categoria = master_df[['id_provincia', 'provincia', 'categoria']].groupby(['provincia','categoria']).count()
display(df_totales_prov_categoria)
values = ['_'.join(row) for row in df_totales_prov_categoria.index]

id_provincia
provincia                                             categoria                                       
Buenos Aires                                          Bibliotecas Populares                        543
                                                      Espacios de Exhibición Patrimonial           152
                                                      Salas de cine                                106
Catamarca                                             Bibliotecas Populares                         38
                                                      Espacios de Exhibición Patrimonial            32
...                                                                                                ...
Tierra del Fuego, Antártida e Islas del Atlántico Sur Espacios de Exhibición Patrimonial            14
                                                      Salas de cine                                  3
Tucumán                                               Bibliotecas Populares                         37
                                                      Espacios de Exhibición Patrimonial            26
                                                      Salas de cine                                  8

[72 rows x 1 columns]

In [58]:
df_resumen3 = pd.DataFrame(
        {
        'etiqueta': ['provincia_categoria']*df_totales_prov_categoria.shape[0],
        'valor': ['_'.join(row) for row in df_totales_prov_categoria.index],
        'total': df_totales_prov_categoria['id_provincia']
    }
)
df_resumen3.reset_index(inplace=True)
df_resumen3.drop(['provincia', 'categoria'], inplace=True, axis='columns')
df_resumen3

,etiqueta,valor,total
0,provincia_categoria,Buenos Aires_Bibliotecas Populares,543
1,provincia_categoria,Buenos Aires_Espacios de Exhibición Patrimonial,152
2,provincia_categoria,Buenos Aires_Salas de cine,106
3,provincia_categoria,Catamarca_Bibliotecas Populares,38
4,provincia_categoria,Catamarca_Espacios de Exhibición Patrimonial,32
...,...,...,...
67,provincia_categoria,"Tierra del Fuego, Antártida e Islas del Atlántico Sur_Espacios de Exhibición Patrimonial",14
68,provincia_categoria,"Tierra del Fuego, Antártida e Islas del Atlántico Sur_Salas de cine",3
69,provincia_categoria,Tucumán_Bibliotecas Populares,37
70,provincia_categoria,Tucumán_Espacios de Exhibición Patrimonial,26


In [63]:
# Concat all three to make the final summary df
df_summary = pd.concat([df_resumen, df_resumen2, df_resumen3], axis='rows', copy=True)
df_summary

,etiqueta,valor,total
0,categoria,Bibliotecas Populares,2017
1,categoria,Espacios de Exhibición Patrimonial,1182
2,categoria,Salas de cine,329
0,fuente,bibliotecas,2017
1,fuente,cines,329
...,...,...,...
67,provincia_categoria,"Tierra del Fuego, Antártida e Islas del Atlántico Sur_Espacios de Exhibición Patrimonial",14
68,provincia_categoria,"Tierra del Fuego, Antártida e Islas del Atlántico Sur_Salas de cine",3
69,provincia_categoria,Tucumán_Bibliotecas Populares,37
70,provincia_categoria,Tucumán_Espacios de Exhibición Patrimonial,26
